In [1]:

import json
import numpy as np
import pandas as pd
import flwr as fl

# ===== 参数：按需改成你的实际设置 =====
SERVER     = "127.0.0.1:8080"   # 如果在同一机器就用这个；否则填服务器IP:端口
TEST_META  = "test_metadata.csv"
MODALITY   = "WSI"              # 可改为 "DNA" / "RNA" / "WSI"
N_CLASSES  = 4                  # 类别数，需与服务端一致
WEIGHT     = 1.0                # 融合权重；若服务端用均值融合可随意

# ===== 读取测试病人列表 =====
meta = pd.read_csv(TEST_META)
assert "patient_id" in meta.columns
pids = meta["patient_id"].astype(str).tolist()

# ===== 演示用“假模型”：生成随机 softmax（Dirichlet）=====
class DummyModel:
    def __init__(self, n_classes: int):
        self.n_classes = n_classes
    def predict_proba(self, x):
        # x 未使用；这里只是演示
        p = np.random.rand(self.n_classes)
        p = np.clip(p, 1e-9, None)
        p = p / p.sum()
        return np.expand_dims(p, 0)  # 形状 [1, C]

dummy_model = DummyModel(N_CLASSES)

# ===== Flower 客户端：在 evaluate(task='predict') 时上传 preds_json =====
class SoftmaxClient(fl.client.NumPyClient):
    def __init__(self, pids, model, modality: str, weight: float):
        self.pids = pids
        self.model = model
        self.modality = modality
        self.weight_for_fusion = float(weight)

    def get_parameters(self, config):
        return []  # 不参与训练，返回空

    def fit(self, parameters, config):
        return [], 0, {}  # 不训练

    def evaluate(self, parameters, config):
        task = config.get("task", "")
        metrics = {}
        if task == "predict":
            rows = []
            for pid in self.pids:
                probs = self.model.predict_proba(None)[0].astype(float)
                probs = np.clip(probs, 1e-9, 1.0)
                probs = probs / probs.sum()
                rows.append({
                    "patient_id": pid,
                    "probs": probs.tolist(),
                    "modality": self.modality,
                    "weight": self.weight_for_fusion,
                })
            metrics = {"preds_json": json.dumps(rows).encode("utf-8")}
        # loss=0.0; 返回样本数用于日志
        return 0.0, len(self.pids), metrics




In [3]:
# ===== 启动客户端，连接服务器并上传 =====
client = SoftmaxClient(pids, dummy_model, MODALITY, WEIGHT)
fl.client.start_numpy_client(server_address=SERVER, client=client)

	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message 39eb5149-fbbe-4d0a-b062-708c2730dd8a
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 746be729-7